### Import librairies

In [18]:
%reload_ext autoreload
%autoreload 0


In [95]:
import os
import pandas as pd
import anndata as ad
from FlowCytometryTools import FCMeasurement
import re
import numpy as np
#from list_paths import create_list_of_paths,create_list_of_paths_spec_patients
from plot_pred_vs_true import plot_result

### Create functions to load and concatenate data

In [35]:
def arcsinh_transform(X,cofactor=5):
    return np.arcsinh(X/cofactor)

In [90]:
def extract_patient_from_path(file_path):
    """extract patient id from file name (works for DOMS, to adapts for others)"""
    file_name = os.path.basename(file_path)
    file_name_without_ext = os.path.splitext(file_name)[0]
    patient_id = file_name_without_ext.split("_")[1]
    return patient_id

In [82]:
def load_data_fcs(path,condition_name,cell_type=None):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)
    anndata.obs_names_make_unique()
    anndata.obs['drug']= condition_name
    
    if cell_type:
        anndata.obs['cell_type']= cell_type
    patient_id = extract_patient_from_path(path)
    anndata.obs['patient'] = patient_id
    anndata.X=arcsinh_transform(anndata.X)
    return anndata

In [4]:
def concatenate_2conditions_data(path_stim,stim_name,path_unstim,unstim_name,outdir_path):
    unstim_anndata = load_data_fcs(path_unstim,unstim_name)
    stim_anndata = load_data_fcs(path_stim,stim_name)

    combined_anndata = stim_anndata.concatenate(unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    combined_anndata.write(outdir_path)
    return

In [38]:
def concatenate_2conditions_multiple_data(path_stim_list,stim_name,path_unstim_list,unstim_name,outdir_path):
    comb_unstim_anndata = load_data_fcs(path_unstim_list[0][0],unstim_name,path_unstim_list[0][1])
    comb_stim_anndata = load_data_fcs(path_stim_list[0][0],stim_name,path_stim_list[0][1])

    for path_stim in path_stim_list[1:]:
        if path_stim[1]:
            stim_anndata = load_data_fcs(path_stim[0],stim_name,path_stim[1])
        else:
            stim_anndata = load_data_fcs(path_stim[0],stim_name)
        assert isinstance(comb_stim_anndata, ad.AnnData), f"Expected AnnData, got {type(comb_stim_anndata)}"
        assert isinstance(stim_anndata, ad.AnnData), f"Expected AnnData, got {type(stim_anndata)}"
        comb_stim_anndata.obs.reset_index(drop=True, inplace=True)
        stim_anndata.obs.reset_index(drop=True, inplace=True)

        # Concaténer les deux objets AnnData
        comb_stim_anndata = ad.concat([comb_stim_anndata, stim_anndata], join='outer', axis=0)
        #comb_stim_anndata = comb_stim_anndata.concatenate(stim_anndata)
        
    for path_unstim in path_unstim_list[1:]:
        if path_unstim[1]:
            unstim_anndata = load_data_fcs(path_unstim[0],unstim_name,path_unstim[1])
        else:
            unstim_anndata = load_data_fcs(path_unstim[0],unstim_name)
        comb_unstim_anndata = ad.concat([comb_unstim_anndata, unstim_anndata], join='outer', axis=0)
        #comb_unstim_anndata = comb_unstim_anndata.concatenate(unstim_anndata)
    comb_stim_anndata.var = comb_stim_anndata.var.astype(str)
    comb_unstim_anndata.var = comb_unstim_anndata.var.astype(str)

    combined_anndata = ad.concat([comb_unstim_anndata, comb_stim_anndata], join='outer', axis=0)
    #combined_anndata.write(outdir_path)
    combined_anndata.write_h5ad(outdir_path)

    return 

In [8]:
def concatenate_1condition_multiple_data(path_list,condition_name,outdir_path):
    comb_anndata = load_data_fcs(path_list[0],condition_name)

    for path in path_list[1:]:
        new_anndata = load_data_fcs(path,condition_name)
        comb_anndata = comb_anndata.concatenate(new_anndata)
    comb_anndata.write(outdir_path)
    return 

In [6]:
def create_list_of_paths2(
    directory, stimulation, cell_type=None, sample=None, patient_excluded=[]
):
    paths_list = []
    for filename in os.listdir(directory):
        if len(patient_excluded) > 0:
            for patient in patient_excluded:
                if (
                    stimulation in filename
                    and (cell_type is None or cell_type in filename)
                    and (sample is None or sample in filename)
                    and patient not in filename
                ):
                    paths_list.append([os.path.join(directory, filename),cell_type])
        else:
            if (
                stimulation in filename
                and (cell_type is None or cell_type in filename)
                and (sample is None or sample in filename)
            ):
                paths_list.append([os.path.join(directory, filename),cell_type])
    return paths_list

In [22]:
def create_list_of_paths_spec_patients(
    directory, stimulation, cell_type=None, sample=None, patient=None
):
    paths_list = []
    for filename in os.listdir(directory):
        if (
            stimulation in filename
            and (cell_type is None or cell_type in filename)
            and (sample is None or sample in filename)
            and (patient is None or patient in filename)
        ):
            paths_list.append([os.path.join(directory, filename),cell_type])
    return paths_list

###  Examples

In [7]:
perio_stim_list=['TNFa','P. gingivalis','IFNa']
perio_cell_list=['Granulocytes (CD45-CD66+)','B-Cells (CD19+CD3-)','Classical Monocytes (CD14+CD16-)','MDSCs (lin-CD11b-CD14+HLADRlo)','mDCs (CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate Monocytes (CD14+CD16+)','Non-classical Monocytes (CD14-CD16+)','CD56+CD16- NK Cells','CD56loCD16+NK Cells','NK Cells (CD7+)','CD4 T-Cells','Tregs (CD25+FoxP3+)','CD8 T-Cells','CD8-CD4- T-Cells']

In [56]:
correspondence = {}
correspondence['Granulocytes (CD45-CD66+)'] ='Neutrophils'
correspondence['B-Cells (CD19+CD3-)'] ='B-cells'
correspondence['Classical Monocytes (CD14+CD16-)'] ='cMCs'
correspondence['MDSCs (lin-CD11b-CD14+HLADRlo)'] ='MDSCs'
correspondence['mDCs (CD11c+HLADR+)'] ='mDCs'
correspondence['pDCs(CD123+HLADR+)'] ='pDCs'
correspondence['Intermediate Monocytes (CD14+CD16+)'] ='intMCs'
correspondence['Non-classical Monocytes (CD14-CD16+)'] ='ncMCs'
correspondence['CD56+CD16- NK Cells'] ='CD16- CD56+ NK'
correspondence['CD56loCD16+NK Cells'] ='CD16+ CD56lo NK'
correspondence['NK Cells (CD7+)'] ='NK'
correspondence['CD4 T-Cells'] ='CD4 T-cells'
correspondence['Tregs (CD25+FoxP3+)'] ='Tregs'
correspondence['CD8 T-Cells'] ='CD8 T-cells'
correspondence['CD8-CD4- T-Cells'] ='CD4- CD8- T-cells'

# Affichage du dictionnaire obtenu
print(correspondence)


{'Granulocytes (CD45-CD66+)': 'Neutrophils', 'B-Cells (CD19+CD3-)': 'B-cells', 'Classical Monocytes (CD14+CD16-)': 'cMCs', 'MDSCs (lin-CD11b-CD14+HLADRlo)': 'MDSCs', 'mDCs (CD11c+HLADR+)': 'mDCs', 'pDCs(CD123+HLADR+)': 'pDCs', 'Intermediate Monocytes (CD14+CD16+)': 'intMCs', 'Non-classical Monocytes (CD14-CD16+)': 'ncMCs', 'CD56+CD16- NK Cells': 'CD16- CD56+ NK', 'CD56loCD16+NK Cells': 'CD16+ CD56lo NK', 'NK Cells (CD7+)': 'NK', 'CD4 T-Cells': 'CD4 T-cells', 'Tregs (CD25+FoxP3+)': 'Tregs', 'CD8 T-Cells': 'CD8 T-cells', 'CD8-CD4- T-Cells': 'CD4- CD8- T-cells'}


In [76]:
c=0
tot=len(perio_stim_list)*len(perio_cell_list)
for cell_type in perio_cell_list:
    for stim in perio_stim_list:
        path_unstim_list=create_list_of_paths_spec_patients(directory='PerioStudy_Complete Plates 1-13_011717 (Clone) - Gated Populations',stimulation='Unstim',cell_type=cell_type,patient='PO1 ')
        path_stim_list=create_list_of_paths_spec_patients(directory='PerioStudy_Complete Plates 1-13_011717 (Clone) - Gated Populations',stimulation=stim,cell_type=cell_type,patient='PO1 ')
        res=concatenate_2conditions_multiple_data(path_stim_list,stim,path_unstim_list,'Unstim',"unstim_perio/perio_data_sherlock"+stim+"_"+cell_type+"_PO1.h5ad")
        c+=1
        print('done ',c,'/',tot)

/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/c

done  1 / 45
done  2 / 45


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/c

done  3 / 45
done  4 / 45
done  5 / 45
done  6 / 45
done  7 / 45
done  8 / 45
done  9 / 45
done  10 / 45
done  11 / 45
done  12 / 45
done  13 / 45
done  14 / 45
done  15 / 45
done  16 / 45


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/c

done  17 / 45
done  18 / 45
done  19 / 45
done  20 / 45
done  21 / 45
done  22 / 45
done  23 / 45
done  24 / 45
done  25 / 45
done  26 / 45
done  27 / 45
done  28 / 45
done  29 / 45
done  30 / 45
done  31 / 45
done  32 / 45
done  33 / 45


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-package

done  34 / 45
done  35 / 45
done  36 / 45
done  37 / 45
done  38 / 45
done  39 / 45
done  40 / 45


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/c

done  41 / 45
done  42 / 45
done  43 / 45
done  44 / 45
done  45 / 45


In [ ]:
doms_stim_list=['LPS','IFNa']
doms_cell_list=['Granulocytes (CD45-CD66+)','B-Cells (CD19+CD3-)','Classical Monocytes (CD14+CD16-)','MDSCs (lin-CD11b-CD14+HLADRlo)','mDCs (CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate Monocytes (CD14+CD16+)','Non-classical Monocytes (CD14-CD16+)','CD56+CD16- NK Cells','CD56loCD16+NK Cells','NK Cells (CD7+)','CD4 T-Cells','Tregs (CD25+FoxP3+)','CD8 T-Cells','CD8-CD4- T-Cells']

In [70]:
markers_doms=['149Sm_pCREB','150Nd_pSTAT5','151Eu_pp38','153Eu_pSTAT1','154Sm_pSTAT3','155Gd_pS6','159Tb_pMAPKAPK2','167Er_pERK','168Er_pSTAT6','169Tm_CD25','174Yb_HLADR','164Dy_IkB','166Er_pNFkB']

In [72]:
ad.read('perio_data_sherlock2/perio_data_sherlockIFNa_CD4 T-Cells_train.h5ad').var_names

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Index(['Time', 'Event_length', '102Pd_BC1', '104Pd_BC2', '105Pd_BC3',
       '106Pd_BC4', '108Pd_BC5', '110Pd_BC6', '113In_CD235ab_CD61',
       '115In_CD45', '139La_CD66', '141Pr_CD7', '142Nd_CD19', '143Nd_CD45RA',
       '144Nd_CD11b', '145Nd_CD4', '146Nd_CD8a', '147Sm_CD11c', '148Nd_CD123',
       '149Sm_CREB', '150Nd_STAT5', '151Eu_p38', '152Sm_TCRgd', '153Eu_STAT1',
       '154Sm_STAT3', '155Gd_S6', '156Gd_CD24', '157Gd_CD38', '158Gd_CD33',
       '159Tb_MAPKAPK2', '160Gd_Tbet', '161Dy_cPARP', '162Dy_FoxP3',
       '164Dy_IkB', '165Ho_CD16', '166Er_NFkB', '167Er_ERK', '168Er_pSTAT6',
       '169Tm_CD25', '170Er_CD3', '171Yb_CD27', '172Yb_CD15', '173Yb_CCR2',
       '174Yb_HLADR', '175Lu_CD14', '176Lu_CD56', '191Ir_DNA1', '193Ir_DNA2',
       'beadDist'],
      dtype='object')

In [84]:
res=ad.read('doms_concatenated/doms_concatenated_IFNa_B-Cells (CD19+CD3-).h5ad')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [83]:
c=0
tot=len(doms_stim_list)*len(doms_cell_list)
for cell_type in doms_cell_list:
    dom_name=correspondence[cell_type]
    for stim in doms_stim_list:
        path_unstim_list=create_list_of_paths2(directory='DOMS_Helios_012025 - Gated Populations',stimulation='Unstim',cell_type=dom_name)
        path_stim_list=create_list_of_paths2(directory='DOMS_Helios_012025 - Gated Populations',stimulation=stim,cell_type=dom_name)
        if len(path_stim_list)==0:
            print(cell_type,stim)
            break
        res=concatenate_2conditions_multiple_data(path_stim_list,stim,path_unstim_list,'Unstim',"doms_concatenated/doms_concatenated_"+stim+"_"+cell_type+".h5ad")
        c+=1
        print('done ',c,'/',tot)


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", I

done  1 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificat

done  2 / 30
done  3 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, ca

...

done  27 / 30


... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.p

done  28 / 30
done  29 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical


done  30 / 30


In [9]:
path_unstim_pred=create_list_of_paths_spec_patients(directory='datasets/PTB drug screen complete - Gated Populations',stimulation='Unstim',cell_type='cMC',patient='HV01')
path_stim_pred=create_list_of_paths_spec_patients(directory='datasets/PTB drug screen complete - Gated Populations',stimulation='LPS',cell_type='cMC',patient='HV01')
res=concatenate_2conditions_multiple_data(path_stim_pred,'LPS',path_unstim_pred,'Unstim',"datasets/PTB_training/LPS_cMC_HV01.h5ad")

/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'drug' as categorical
... storing 'cell_type' as categorical


### Pipeline for evaluation

In [2]:
EVALUATION_MARKERS = [
    '151Eu_pp38', '155Gd_pS6', '149Sm_pCREB', 
    '159Tb_pMAPKAPK2', '166Er_pNFkB', '167Er_pERK12'
]

In [ ]:
PTB_EVALUATION_MARKERS = ['151Eu_pP38', '155Gd_pS6', '149Sm_pCREB', '159Tb_pMK2', '166Er_pNFkB', '167Er_pERK']

train on CMC, pp38, pS6, LPS

In [148]:
original_path = "datasets/PTB_training/combined_LPS_cMC.h5ad"
target = ad.read(original_path)
target1 = target[:, '151Eu_pP38'].copy()

In [154]:
target1[target1.obs['condition'] == 'control'].X.flatten()

ArrayView([2.0405424, 1.9779768, 0.6029839, ..., 0.8639047, 0.902362 ,
           1.681758 ], dtype=float32)

In [ ]:
for marker in PTB_EVALUATION_MARKERS:
    plot_result('results/PTB/LPS_cMC/model-cellot/pred.csv','datasets/PTB_training/LPS_cMC_HV01.h5ad',marker,f"results/PTB/LPS_cMC/model-cellot/unseen_plot/LPS_cMC_HV01_{marker}.png")

In [101]:
np.mean(ad.read('unstim_perio/perio_data_sherlockIFNa_B-Cells (CD19+CD3-)_PO1.h5ad')[:,'159Tb_MAPKAPK2'][ad.read('unstim_perio/perio_data_sherlockIFNa_B-Cells (CD19+CD3-)_PO1.h5ad')[:,'159Tb_MAPKAPK2'].obs['drug']=='Unstim'].X.flatten())

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


ArrayView(2.0665224, dtype=float32)

In [103]:
plot_result('./pred.csv','perio_data_sherlock2/perio_data_sherlockIFNa_B-Cells (CD19+CD3-)_train.h5ad','159Tb_MAPKAPK2','./test_perio.png','IFNa')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
